In [ ]:
pip install ultralytics


In [ ]:
pip install gradio

In [ ]:
import cv2
from google.colab import drive
import os

In [ ]:


# # Path to the dataset folder
# path = '/content/drive/My Drive/Traffic and Road Signs.v1i.yolov5pytorch'

# # List of the files in the folder
# files = os.listdir(path)
# print("Files in the folder:", files)

# # 'data.yaml' check
# if 'data.yaml' in files:
#     print("data.yaml found!")
# else:
#     print("data.yaml is missing from the folder. Please ensure it is in the correct location.")


In [ ]:
# from ultralytics import YOLO

# #pre trained yolo model
# model = YOLO('yolov8n.pt')

# #training the model
# data_yaml_path = '/content/drive/My Drive/Traffic and Road Signs.v1i.yolov5pytorch/data.yaml'


# model.train(data=data_yaml_path, epochs=1, imgsz=640)

# # Evaluate the model
# results = model.val()


# model.export(format='onnx')


In [ ]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
import gradio as gr

# Loading model best1 and best2
model1 = YOLO('best2.pt')
model2 = YOLO('best1.pt')



In [ ]:

#this function will process the image and predict whether there exist road sign in image or not
def process_image(image):


    results1 = model1(image)
    annotated_image1 = results1[0].plot()

    # Convert BGR to RGB for proper colors as RGB is more compatible than BRG
    annotated_image1 = cv2.cvtColor(annotated_image1, cv2.COLOR_BGR2RGB)


    results2 = model2(image)
    annotated_image2 = results2[0].plot()


    annotated_image2 = cv2.cvtColor(annotated_image2, cv2.COLOR_BGR2RGB)

    return annotated_image1, annotated_image2


In [ ]:
def process_video(video_path):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return "Error: Couldn't open video."

    # creating a folder where videos output can be saved if folder dosent exsit os.makedirs() will make the folder for us
    temp_dir = "./temp"
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    #outout_model1.mp4 and output_model2.mp4 is name of of output videos
    output_video_path1 = os.path.join(temp_dir, "output_model1.mp4")
    output_video_path2 = os.path.join(temp_dir, "output_model2.mp4")

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    out1 = cv2.VideoWriter(output_video_path1, fourcc, fps, (frame_width, frame_height))
    out2 = cv2.VideoWriter(output_video_path2, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break


        results1 = model1.predict(frame)
        annotated_frame1 = results1[0].plot()
        out1.write(annotated_frame1)

        # Inference with the second model
        results2 = model2.predict(frame)
        annotated_frame2 = results2[0].plot()
        out2.write(annotated_frame2)

    cap.release()
    out1.release()
    out2.release()

    return output_video_path1, output_video_path2


In [ ]:
# Main function to handle image and video input
def process_input(input_path, output_folder):

    if os.path.isfile(input_path):
        file_ext = os.path.splitext(input_path)[-1].lower()
        if file_ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            process_image(input_path, output_folder)
        elif file_ext in ['.mp4', '.avi', '.mov', '.mkv']:
            process_video(input_path, output_folder)
        else:
            print("Unsupported file format!")
    else:
        print("Error: Input file does not exist.")


In [ ]:

#code for front end using gradio
def gradio_interface(input_file):
    file_ext = os.path.splitext(input_file.name)[-1].lower()
    if file_ext in ['.jpg', '.jpeg', '.png', '.bmp']:
        # Process image
        image = cv2.imread(input_file.name)
        annotated_image1, annotated_image2 = process_image(image)
        return annotated_image1, annotated_image2
    elif file_ext in ['.mp4', '.avi', '.mov', '.mkv']:
        # Process video
        video_path1, video_path2 = process_video(input_file.name)
        return video_path1, video_path2
    else:
        return "Unsupported file format!"

In [ ]:

# defining the gradio application
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.File(label="Upload Image or Video"),
    outputs=[
        gr.Image(label="Model 1 Annotated Image"),
        gr.Image(label="Model 2 Annotated Image"),
    ],
    title="YOLOv8 Dual Model Prediction",
    description="Upload an image or video to see predictions from two YOLOv8 models."
)

# launching the application
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://486300be7bd224a556.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#this code is for live time predection for image and videos predection code is present above

import cv2
from ultralytics import YOLO

# Step 1: Load YOLO models
model_path1 = 'best2.pt'  
model_path2 = 'best1.pt'  
model1 = YOLO(model_path1) 
model2 = YOLO(model_path2)  


cap = cv2.VideoCapture(0) 

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

while True:

    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    
    results1 = model1.predict(frame)  
    annotated_frame1 = results1[0].plot() 

    
    results2 = model2.predict(frame)  
    annotated_frame2 = results2[0].plot()  

    
    combined_frame = cv2.hconcat([annotated_frame1, annotated_frame2])


    cv2.imshow('Real-Time YOLO Prediction', combined_frame)

    
    if cv2.waitKey(1) & 0xFF == 27:
        break


cap.release()
cv2.destroyAllWindows()
